In [11]:
import re
import sys
import time
import csv
from datetime import datetime
import random
import requests

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import os
import pandas as pd
import numpy as np
import importlib
os.chdir('..') # from scraping-notebooks -> web-scrapping
sys.path.append(os.path.join(os.getcwd() + '\scraping-py')) # append the scraping-py folder to path
from helper_functions import get_user_agent, prep_driver


ModuleNotFoundError: No module named 'random_user_agent'

In [24]:
helper_path = os.path.join(os.getcwd(), 'data')
print(helper_path)
df = pd.read_csv(helper_path + '\dollskill_urls.csv', index_col = 0)
df.head(3)

c:\Users\linds\OneDrive\chngingroom\web-scrapping\data


,product_url,image_links
display_name,,
Chain Smoker Set,https://www.dollskill.com/chain-smoker-set.html,['https://media.dollskill.com/media/9SGF9tQCsc...
St Marine Westenders,https://www.dollskill.com/one-teaspoon-st-mari...,['https://media.dollskill.com/media/dUd4Ukn86u...
Young And Beautiful Bodysuit,https://www.dollskill.com/white-lace-bodysuit....,['https://media.dollskill.com/media/VCdVTk75MD...


In [3]:
df.shape

(139126, 3)

In [4]:
df['image_links'].iloc[0]
one_row = df.iloc[0]

In [122]:
user_agent: str = get_user_agent()


def get_product_info(row):
	start_time = time.time()
	url = row['product_url']
	options = prep_driver(user_agent)
	driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
	driver.get(url)
	print('product url:', url)

	# get `size`:
	size_path = '//main/div/div[1]/div/div/div/div[2]/div/div[7]/select/option'
	size_els = driver.find_elements(By.XPATH, size_path)
	size = []
	if len(size_els) == 1:
		# one size item
		size.append(size_els[0].text)
	else:
		# multiple sized item
		# regex only extracts size & not whether out of stock
		size = [re.match('^[^-]*[^ -]', el.text).group(0) for el in size_els if 'select a size' not in el.text.lower()]
	print('size:', size)

	# get `price`: current price (sale price if on sale)
	price_path = '//main/div/div[1]/div/div/div/div[2]/div/div[3]/div/span'
	price_els = driver.find_elements(By.XPATH, price_path)
	price = re.search('[-0-9.,]+', price_els[-1].text).group()
	print('price:', price)

	# get `availability`: if sold out
	avail_path = '//main/div/div[1]/div/div/div/div[1]/div/div/div/div[2]/div'
	avail_el = driver.find_element(By.XPATH, avail_path)
	availability = 1
	if avail_el.text.lower() == 'sold out':
		availability = 0
	print(availability)

	# get everything drop dropdown buttons
	# 1. find "Description" button
	buttons_xpath = '//div[1]/main/div/div[1]/div/div/div/div[2]/div/div[10]/'
	descrip_xpath = buttons_xpath + 'div[1]'
	# descrip = driver.find_element(By.XPATH, "//*[text()='Description']")
	# buttons_parent = descrip.find_element(By.XPATH, '../../../..') # div holding 4 drop-down buttons
	# descrip_button = descrip.find_element(By.XPATH, '../../..')
	descrip_button = driver.find_element(By.XPATH, descrip_xpath)
	desc_button_class = descrip_button.get_attribute('class')
	# print('class:', desc_button_class)
	driver.execute_script("arguments[0].scrollIntoView();", descrip_button)
	driver.execute_script("arguments[0].click();", descrip_button)
	description = descrip_button.find_element(By.TAG_NAME, 'p').text
	print('description:', description)

	# get `brand`
	brand_path = '//main/div/div[1]/div/div/div/div[2]/div/div[1]/a'
	brand_el = driver.find_element(By.XPATH, brand_path)
	# Code to continue searching for brand if element DNE:
	# list_path = os.path.join(os.getcwd(), 'web-scrapping', 'data', 'dollskill_brands.npy')
	# print(list_path)
	# brand_list = loadList(list_path)
	# if brand_el.text == '':
	# 	# brand element not here, check if in description
	# 	brand_match = ''
	# 	for brand in brand_list:
	# 		if brand in description:
	# 			brand_match = brand
	# 			print('found a match the 2nd time:', brand_match)
	# else:
	# 	print('brand:', brand_el.text)
	
	brand = 'Dolls Kill' # default brand
	if len(brand_el.text) != 0:
		brand = brand_el.text

	# 2. Find "Details" button
	details_path = buttons_xpath + 'div[2]'
	details_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, details_path)))
	details_button.click()
	# print('details class:', details_button.get_attribute('class'))
	bullet_pt_els = details_button.find_elements(By.XPATH, './/ul/li/span')
	for bullet in bullet_pt_els:
		print(bullet.text)

	# # 3. Find "View More" button (4th button)
	# view_path = buttons_xpath + 'div[4]'
	# view_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, view_path)))
	# view_button.click()
	# category_els = view_button.find_elements(By.XPATH, './/ul/a')
	# for cat in category_els:
	# 	print(cat.text)

In [123]:
# testing for 3 products
row = df.iloc[0]
row1 = df.iloc[1]
row2 = df.iloc[10000]
get_product_info(row)
get_product_info(row1)
ul_el = get_product_info(row2)
# print(ul_el)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\linds\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-122-ecedaca97171>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)


product url: https://www.dollskill.com/chain-smoker-set.html
description: Chain Smoker Set is burnin’ hot. This sexy set includes a high-neck cropped top with long sheer sleeves and shorts with flowing sheer panels.
c:\Users\linds\OneDrive\chngingroom\web-scrapping\data\dollskill_brands.npy




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\linds\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


product url: https://www.dollskill.com/one-teaspoon-st-marine-westenders.html
description: One Teaspoon St Marine Westenders are perfect for late nights in tha studio and the early mornin’ classes that follow. Slide into these incredible super low rise jeans, featurin’ a washed premium denim construction, slouchy straight leg cut that flares slightly, hand distressed ‘n patched details all over, and classic zipper fly and button closure.
c:\Users\linds\OneDrive\chngingroom\web-scrapping\data\dollskill_brands.npy
brand: One Teaspoon




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\linds\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


product url: https://www.dollskill.com/coca-cola-logo-midi-skirt.html
description: Coca Cola Logo Midi Skirt that pop and fizz is what we are all about. No wonder, because when yer wearin this exxxtra tight midi skirt, we wouldnt have it any other way. Featurin fabric that loves yer curves as much as we do and an elastic band that keeps it up, we are more than ready to take a sip of you. With its logo that is a total throwback to yer favorite hot day drink, you look a tall glass of something delicious.
c:\Users\linds\OneDrive\chngingroom\web-scrapping\data\dollskill_brands.npy


In [ ]:
# checked: product urls are all unique
display_names = df['display_name'].value_counts().rename_axis('display_name').reset_index(name='count')
display_names.shape
# non_unique_names = display_names[(display_names.iloc[:, 1] > 1)]

(138546, 2)

In [134]:
"dollskill-" + "-".join(row2['display_name'].split())
type(df.shape[0])

int

In [139]:
### id_style						? do I need to scrape - interesting test cases
# display_name: DONE
# materials							? is there a regex for this
# color
# size: DONE
# price: DONE
### currency: USD
# product_url: DONE
# image_links: DONE
# brand_name: DONE
### retailer: Dolls Kill
# description: DONE
### scrapped_date: today's date
# high_level						need to talk to Sarah about how she's been doing this
# low_level							need to talk to Sarah
### gender: Women					?
### secondhand: 0
### shipping_from: np.NaN
# style								? is there regex for this
# availability: DONE

In [140]:
dictionary = {'size' : "sizes", 'price' : "prices", 'description' : "descriptions",
					'brand' : "brands", 'availability' : "availabilities", 'gender' : "gender",
					'secondhand' : "secondhand", 'shipping_from' : "shipping_from",
					'scrapped_date' : "scrapped_date", 'retailer' : "retailer", 'currency' : "currency"}

In [14]:
# experimenting with none types in csv & saving shit
listofNones = [TypeError, 0, 1, None]
df = pd.DataFrame(listofNones, columns=['avail'])
df.to_csv('test.csv', index=False)

In [15]:
df_read = pd.read_csv('test.csv')
df_read.head(5)

,avail
0,<class 'TypeError'>
1,0
2,1
3,NaN


In [22]:
df_read['avail'].loc[0]

"<class 'TypeError'>"

In [23]:
# experimenting with try except
try:
	None + None
except Exception as excep:
	print(str(type(excep)))
	print(excep)

<class 'TypeError'>
unsupported operand type(s) for +: 'NoneType' and 'NoneType'
